# How to Segment Buildings on Drone Imagery with Fast.ai & Cloud-Native GeoData Tools

From this [blog post](https://medium.com/@anthropoco/how-to-segment-buildings-on-drone-imagery-with-fast-ai-cloud-native-geodata-tools-ae249612c321) and [notebook](https://nbviewer.jupyter.org/github/daveluo/zanzibar-aerial-mapping/blob/master/geo_fastai_tutorial01_public_v1.ipynb). The Colab Notebook is [here](https://colab.research.google.com/github/daveluo/zanzibar-aerial-mapping/blob/master/geo_fastai_tutorial01_public_v1.ipynb#scrollTo=eOA6ZD9KJoQz).

Key Steps:
* Preprocess image geoTIFFs and manually labeled data geoJSON files into training data for deep learning
* Create a U-net segmentation model to predict what pixels in an image represent buildings (and building-related features)
* Test our model's performance on unseen imagery with GPU or CPU
* Post-process raw model outputs into geo-registered building shapes evaluated against ground turch

## Installing Packages/Dependencies

In [1]:
import time
start_time = time.time()

# Install conda and add channels to look for packages in
# Adapted from: https://stackoverflow.com/questions/62417114/how-do-i-install-anaconda-in-google-colab
# Assumes we are using Python 3.7 in Colab
import sys
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append('/usr/local/lib/python3.7/site-packages/')
!conda update -n base -c defaults conda -y
!conda config --add channels conda-forge
!git clone https://github.com/cosmiq/solaris.git


environment_yml = "/content/solaris/environment.yml"

with open(environment_yml, 'r') as f:
    file_source = f.read()

replace_string = file_source.replace("solaris", "base")
shapely_old = "git+https://github.com/Toblerity/shapely.git@master#egg=shapely-1.7.1dev"
shapely_new = "git+https://github.com/Toblerity/shapely.git@master"
replace_string = replace_string.replace(shapely_old, shapely_new)

with open(environment_yml, 'w') as f:
    f.write(replace_string)

!conda env update -f /content/solaris/environment.yml
!sudo apt install libspatialindex-dev
!pip install ./solaris/.

metrics_py = "/usr/local/lib/python3.7/site-packages/solaris/nets/metrics.py"
with open(metrics_py, 'r') as f:
    file_source = f.read()

replace_string = file_source.replace("'cosine':", "#'cosine':")
replace_string = replace_string.replace("'cosine_proximity':",
                                        "#'cosine_proximity':")

with open(metrics_py, 'w') as f:
    f.write(replace_string)

keras_losses_py = "/usr/local/lib/python3.7/site-packages/solaris/nets/_keras_losses.py"
with open(keras_losses_py, 'r') as f:
    file_source = f.read()

replace_string = file_source.replace("'cosine':", "#'cosine':")

with open(keras_losses_py, 'w') as f:
    f.write(replace_string)

total_time = time.time() - start_time
print(f"It took {total_time/60} minutes to install packages/dependencies.")

--2021-03-07 19:37:21--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90040905 (86M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.9.2-Linux-x86_64.sh’

Miniconda3-py37_4.9 100%[===================>]  85.87M   130MB/s    in 0.7s    

2021-03-07 19:37:22 (130 MB/s) - ‘Miniconda3-py37_4.9.2-Linux-x86_64.sh’ saved [90040905/90040905]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py37h261ae71_2
    - chardet==3.0.4=py37h06a4308_1003
    - conda-packa

## Pre-Processing

In [2]:
import solaris as sol
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
from pathlib import Path
import rasterio
import os